In [1]:
import gymnasium as gym
from stable_baselines3 import PPO



In [2]:
# model = PPO("MlpPolicy", "CartPole-v1").learn(10_000)
env = gym.make("Pong-v0", render_mode="rgb_array")


/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [ ]:

# Create the Pong environment

# Train a PPO model (You can adjust timesteps based on your computational resources)
model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=100000)  # Train for 100,000 steps

# Save the model
model.save("ppo_pong")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


2024-09-20 14:07:48.721 Python[52735:3823944] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [3]:
# Load the saved PPO model
model = PPO.load("ppo_pong")


In [4]:
from transformers import pipeline
import tqdm as notebook_tqd
# Load a pre-trained language model from Hugging Face (GPT-2 as an example)
llm = pipeline("text-generation", model="gpt2")

# Function to convert game state to text (very basic conversion)
def describe_game_state(obs):
    # For simplicity, describe game state in terms of paddle position and ball direction
    # You can design this function to convert the observation (obs) into human-readable text
    return "The ball is moving towards the left side. Your paddle is in the middle."

# Function to ask the LLM for advice based on the game state
def get_llm_advice(game_state_description):
    response = llm(game_state_description, max_length=50, num_return_sequences=1)
    advice = response[0]['generated_text']
    return advice

# Function to interpret LLM advice into action
def llm_to_action(advice):
    if "move paddle down" in advice:
        return 2  # Action for moving the paddle down
    elif "move paddle up" in advice:
        return 3  # Action for moving the paddle up
    else:
        return 0  # Default action (e.g., no movement)


/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kpradjinata/Documents/Research/OpenAIGym/LLM/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [6]:
# Reset the environment and unpack the observation (ignoring the info)
obs, info = env.reset()  # Gymnasium returns a tuple (obs, info)

for i in range(10000):
    # Get the RL model's prediction for the action using just the observation
    rl_action, _states = model.predict(obs)
    
    # Step the environment using the action; note the five returned values
    obs, reward, done, truncated, info = env.step(rl_action)
    # print(reward,done)
    if reward != 0:
        print(obs)
    # Render the environment (show the game)
    env.render()

    # If the episode ends, reset the environment again
    if done or truncated:
        obs, info = env.reset()

env.close()


[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 ...

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]]
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 ...

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ..

KeyboardInterrupt: 